In [ ]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"Implicit", category=UserWarning)
warnings.filterwarnings("ignore", message=r"cannot", category=UserWarning)

In [ ]:
import json
import os
import sys
sys.path.append('..')

from model_shifts import plot_experiment

In [ ]:
def process_experiment_data(experiment_path, experiment_name):
    # filenames = [name for name in os.listdir(experiment_path) if os.path.isfile(os.path.join(f'{experiment_path}/', name))]
    filenames = [name for name in os.listdir(experiment_path)]
    filenames = list(filter(lambda name: name.endswith(experiment_name), filenames))

    def process_dict(data, result, num):
        for k, v in data.items():
            # If v is a dictionary then process it recursively
            # print(result, k)
            if k not in result and isinstance(v, dict):
                result[k] = {}
                result[k] = process_dict(v, result[k], num)

            elif k in result and isinstance(v, dict):
                result[k] = process_dict(v, result[k], num)

            # If v is not a dictionary and the value has not been encountered before, store it
            elif k not in result and (isinstance(v, float) or isinstance(v, int)):
                result[k] = (v / num)

            # If v is in the dictinary and the value has been encountered before, add it
            elif k in result and (isinstance(v, float) or isinstance(v, int)):
                result[k] += (v / num)

        return result

    average_results = {}
    for file in filenames:
        with open(f'{experiment_path}/{file}/measurements.json') as data_file:
            data = json.load(data_file)
            average_results = process_dict(data, average_results, len(filenames))

    return average_results

In [ ]:
path = '../experiment_data'
name = 'German_processed_NN_hidden_5_attempt_2'

result = process_experiment_data(path, name)

os.makedirs(f'{path}/{name}')
with open(f'{path}/{name}/measurements.json', 'w') as outfile:
    json.dump(result, outfile, indent=3)


In [ ]:
config = [
    {'type': 'pos_MMD', 'dict_path': ['MMD', 'positive', 'value']},
    {'type': 'neg_MMD', 'dict_path': ['MMD', 'negative', 'value']},
    {'type': 'pos_distance', 'dict_path': ['decisiveness', 'positive']},
    {'type': 'neg_distance', 'dict_path': ['decisiveness', 'negative']},
    {'type': 'disagreement', 'dict_path': ['disagreement']},
    {'type': 'model_MMD', 'dict_path': ['model_MMD', 'value']},
    {'type': 'sample_MMD', 'dict_path': ['sample_MMD', 'value']}
]
for c in config:
    plot_experiment(f'{path}/{name}',
                    ['DICE_3', 'Wachter'],
                    c['type'],
                    c['dict_path'],
                    show_plot=False)